In [154]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [156]:
from abc import ABC, abstractmethod
from collections import ChainMap
from functools import wraps, update_wrapper
import inspect
from inspect import Parameter, getsource, getsourcelines, getsourcefile, \
    getfile, ismodule, ismethod, isfunction

from htools import *

In [5]:
@contextmanager
def temporary_global_scope(kwargs):
    # Make kwargs temporarily available as global vars.
    print('in ctx', kwargs)
    old_globals = globals().copy()
    globals().update(kwargs)
    try:
        yield
    finally:
        for k in kwargs:
            if k in old_globals:
                globals()[k] = old_globals[k]
            else:
                del globals()[k]

In [139]:
def add_kwargs(*fns, positional=True, variable=True):
    """When multiple functions contain a parameter with the same name, 
    priority is determined by the order of `fns`. Wrapped function must accept
    **kwargs.
    """
    param_types = {Parameter.POSITIONAL_OR_KEYWORD, Parameter.KEYWORD_ONLY}
    if positional: param_types.add(Parameter.POSITIONAL_ONLY)
        
    def _args(fn): 
        res = {}
        for k, v in params(fn).items():
            # If positional=False, allow positional_or_keyword args with
            # defaults but not those without. 
            if v.kind in param_types and (positional 
                                          or v.default != inspect._empty):
                res[k] = v
                
            # args/kwargs are converted to non-varying types and names are
            # adjusted to include function name. E.g. if we're adding kwargs
            # from function foo which accepts kwargs, that arg becomes a 
            # keyword-only dictionary called foo_kwargs.
            elif variable:
                name = f'{fn.__name__}_{k}'
                if v.kind == Parameter.VAR_POSITIONAL:
                    kind = Parameter.POSITIONAL_OR_KEYWORD
                    default = ()
                elif v.kind == Parameter.VAR_KEYWORD:
                    kind = Parameter.KEYWORD_ONLY
                    default = {}
                else:
                    continue
                res[name] = Parameter(name, kind, default=default)
        return res
    
    # ChainMap operates in reverse order so functions that appear earlier in
    # `fns` take priority.
    extras_ = dict(ChainMap(*map(_args, fns)))

    def decorator(func):
        """First get params present in func's original signature, then get
        params from additional functions which are NOT present in original 
        signature. Combine and sort param lists so positional args come first
        etc. Finally replace func's signature with our newly constructed one.
        """
        sig = inspect.signature(func)
        extras = [v for v in 
                  select(extras_, drop=sig.parameters.keys()).values()]
        parameters = sorted(
            list(sig.parameters.values()) + extras, 
            key=lambda x: (x.kind, x.default != inspect._empty)
        )
        func.__signature__ = sig.replace(parameters=parameters)
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            """Execute wrapped function in a context where kwargs are 
            temporarily available as globals. Globals will be restored to 
            its prior state once execution completes.
            """
            # Order matters here: defaults must come first so user-passed 
            # args/kwargs will override them.
            kwargs = {**{p.name: p.default for p in extras},  
                      **func.__signature__.bind(*args, **kwargs).arguments}
            with temporary_global_scope(kwargs):
                return func(**kwargs)
        return wrapper
    return decorator

In [140]:
class CLI(ABC):
    
    def __init__(self, *funcs):
        self.funcs = list(funcs)
        
        @add_kwargs(*self.funcs, positional=True)
        def _call_wrapper(self, **kwargs):
            print(_call_wrapper.__signature__)
            print(Args(a=a, b=b, c=c, d=d))
            return self.call(**kwargs)
        CLI.__call__ = _call_wrapper
        
    @abstractmethod
    def call(self):
        raise NotImplementedError()

In [148]:
class MyCLI(CLI):
    
    # TODO: mostly works but I don't think we support args/kwargs at the 
    # moment. Maybe add them to signature with func name, i.e. bar_kwargs?
    def call(self, **kwargs):
        print('in __call__', kwargs)
        f_res = foo(a=a, b=b)
        b_res = baz(c=c, d=d)
        print(f_res)
        print(b_res)

In [149]:
def foo(a, b=3, **kwargs):
    return 'foo', {'a': a, 'b': b}

def baz(c, *args, d=6):
    return 'baz', {'c': c, 'd': d}

In [150]:
cli = MyCLI(foo, baz)

In [151]:
cli('c0', 'a1', 'd2')

in ctx {'c': 'c0', 'baz_args': 'd2', 'd': 6, 'a': 'a1', 'b': 3, 'foo_kwargs': {}, 'self': <__main__.MyCLI object at 0x107e7d7b8>}
(self, c, a, baz_args=(), b=3, *, d=6, foo_kwargs={}, **kwargs)
Args(a='a1', b=3, c='c0', d=6)
in __call__ {'c': 'c0', 'baz_args': 'd2', 'd': 6, 'a': 'a1', 'b': 3, 'foo_kwargs': {}}
('foo', {'a': 'a1', 'b': 3})
('baz', {'c': 'c0', 'd': 6})


In [110]:
cli = CLI(foo, baz)

In [111]:
cli.bar('x1', 9, d=11, b=44)

in ctx {'c': 'x1', 'd': 11, 'a': 9, 'b': 44, 'self': <__main__.CLI object at 0x1a16171c50>}
(self, c, a, d=6, b=3, **kwargs)
Args(a=9, b=44, c='x1', d=11)
in __call__ {'c': 'x1', 'd': 11, 'a': 9, 'b': 44}
396
x1x1x1x1x1x1x1x1x1x1x1


### Alternate strategies

- Update signature automatically with deco as above, but then explicitly update globals with kwargs. Is this any safer? Maybe not. I guess it removes 1 layer of hackery.
- 

## Picklable

In [163]:
class Picklable:
    
    def test(self):
        return getsource(type(self))

In [165]:
getsource(Picklable)

TypeError: <module '__main__'> is a built-in class

In [164]:
Picklable().test()

TypeError: <module '__main__'> is a built-in class